In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pydaddy
from scipy.signal import correlate
from scipy.optimize import curve_fit
from scipy.special import rel_entr
from scipy.stats import wasserstein_distance
import sdeint

In [ ]:
# plt.rcParams.update(
#     {
#         'font.family': 'sans-serif',
#         'font.sans-serif': 'Helvetica',
#         'font.size': 32,
#     }
# )

In [ ]:
def acf(data, t_lag=1000):
    """
    Calculates autocorrelation using wiener khinchin theorem.
    """

    data = data - data.mean()
    x = np.arange(0, t_lag)
    c = np.fft.ifft(np.square(np.abs(np.fft.fft(data))))
    c /= c[0]
    return c[0:t_lag]

def act(m):
    rho = acf(m)
    rho = rho[rho.argmax():]
    t = np.arange(rho.size)
    ftau = lambda t, a, b, c: a * np.exp((-t / b)) + c
    params, cov = curve_fit(ftau, t, rho)
    tau = params[1]
    return tau

def trel(m1, m2):
    tau1, tau2 = act(m1), act(m2)
    print(f'tau_1: {tau1}')
    print(f'tau_2: {tau2}')    
    
    print(f'T_rel: {abs(tau1 - tau2)/ tau1}')

In [ ]:
def simulate(F1, F2, G11, G22, t_int, timepoints, x0=None):
    tspan = np.arange(0, t_int * timepoints, step=t_int)
    
    def F(x):
        return np.array([F1(*x), F2(*x)])

    def G(x):
            return np.diag([np.sqrt(np.abs(G11(*x))), np.sqrt(np.abs(G22(*x)))])

    if x0 is None:
        x0 = np.array([0., 0.])

    x_sim = np.zeros((timepoints, 2))
    x_sim[0, :] = x0
    
    for i in range(1, timepoints):
        x_next = (x_sim[i - 1, :] + 
                  t_int * F(x_sim[i - 1, :]) + 
                  np.sqrt(t_int) * G(x_sim[i - 1, :]) @ np.random.normal(size=(2, )))
        while(x_next[0] ** 2 + x_next[1] ** 2 > 1):
            x_next = (x_sim[i - 1, :] + 
                  t_int * F(x_sim[i - 1, :]) + 
                  np.sqrt(t_int) * G(x_sim[i - 1, :]) @ np.random.normal(size=(2, )))
        x_sim[i, :] = x_next
        
    return x_sim

In [ ]:
dataset = '30'
f_act = f'data/{dataset}_extracted.npy'
f_sim = f'data/sampled_x_0_bc_{dataset}.npy'

actual = np.load(f_act)
# actual, _ = pydaddy.load_sample_dataset('model-data-vector-ternary')
# actual = np.array(actual).T
simulated = np.load(f_sim)

In [ ]:
actual.shape, simulated.shape

In [ ]:
modm_actual = np.sqrt(actual[:, 0] ** 2 + actual[:, 1] ** 2)
modm_simulated = np.sqrt(simulated[:, 0] ** 2 + simulated[:, 1] ** 2)

In [ ]:
wasserstein_distance(modm_actual, modm_simulated)

In [ ]:
trel(modm_actual, modm_simulated)

In [ ]:
plt.figure(figsize=(8, 8))
plt.hist(np.sqrt(actual[:, 0] ** 2 + actual[:, 1] ** 2), bins=100, density=True, alpha=0.5, label='Actual')
plt.hist(np.sqrt(simulated[:, 0] ** 2 + simulated[:, 1] ** 2), bins=100, density=True, alpha=0.5, label='Simulated')
plt.legend()
plt.xlabel('$|\mathbf{m}|$')
plt.ylabel('$p(|\mathbf{m}|)$')
plt.xlim([0, 1.1])
plt.tight_layout()
plt.savefig(f'{dataset}_histogram.pdf')
plt.show()

In [ ]:
# acf_actual = correlate(modm_actual, modm_actual)
# # acf_actual = acf_actual[acf_actual.argmax():] / acf_actual.max()
# acf_simulated = correlate(modm_simulated, modm_simulated)
# acf_simulated = acf_simulated[acf_simulated.argmax():] #/ acf_simulated.max()

# acf_simulated_pydaddy = correlate(modm_simulated_pyd, modm_simulated_pyd)
# acf_simulated_pydaddy= acf_simulated_pydaddy[acf_simulated_pydaddy.argmax():] #/ acf_simulated_pydaddy.max()

acf_actual = acf(modm_actual)
acf_simulated = acf(modm_simulated)
# acf_simulated_pyd = acf(modm_simulated_pyd)

plt.figure(figsize=(8, 8))
plt.plot(acf_actual[:200], lw=3, label='Actual')
plt.plot(acf_simulated[:200], lw=3, label='Simulated')
# plt.plot(acf_simulated_pyd[:1000], lw=3)
plt.xlabel('$t$')
plt.ylabel('Autocorrelation $\\rho_{|\\mathbf{m}|}$')
plt.legend()
plt.tight_layout()
# plt.savefig(f'{dataset}_autocorr.pdf')
plt.show()

In [ ]:
plt.plot(modm_actual[:1000], label='Actual')
plt.plot(modm_simulated[:1000], label='Simulated')
plt.legend()
plt.xlabel('t')
plt.ylabel('|m|')
plt.show()

In [ ]:
dd = pydaddy.Characterize(actual.T, t_inc=0.12, bins=20)

In [ ]:
print(dd.fit('F1', order=3, threshold=0.005))

In [ ]:
print(dd.fit('F2', order=3, threshold=0.005))

In [ ]:
print(dd.fit('G11', order=2, threshold=0.001))

In [ ]:
print(dd.fit('G22', order=2, threshold=0.001))

In [ ]:
print(dd.fit('G12', order=2, threshold=0.001))

In [ ]:
actual.shape[0]

In [ ]:
simulated_pydaddy = simulate(F1=dd.F1, F2=dd.F2, G11=dd.G11, G22=dd.G22,
    t_int=1, timepoints=actual.shape[0], x0=actual[0, :])
modm_simulated_pyd = np.sqrt(simulated_pydaddy[:, 0] ** 2 + simulated_pydaddy[:, 1] ** 2)

In [ ]:
plt.figure(figsize=(8, 8))
plt.hist(modm_actual, bins=100, density=True, alpha=0.5, label='Actual')
plt.hist(modm_simulated_pyd, range=(0, 1.1), bins=100, density=True, alpha=0.5, label='Simulated (PyDaddy)')
# plt.hist(np.sqrt(simulated[:, 0] ** 2 + simulated[:, 1] ** 2), bins=100, density=True, alpha=0.5, label='Simulated (Neural)')
plt.legend()
plt.xlabel('|m|')
plt.ylabel('p(|m|)')
plt.xlim((0, 1.1))
plt.show()

In [ ]:
print(wasserstein_distance(modm_actual, modm_simulated))
print(wasserstein_distance(modm_actual, modm_simulated_pyd))

In [ ]:
trel(modm_actual, modm_simulated)
trel(modm_actual, modm_simulated_pyd)